In [20]:
# -- Enhancer Agent --
# This script sets up an agent that enhances queries by extracting structured metadata and filters.
# It uses a set of tools to analyze and transform vague or unstructured queries into clear, structured metadata.
# The agent is designed to work with the LangChain framework and utilizes OpenAI's GPT-4 model.
# The agent is capable of using various tools such as keyword extractors, numeric constraint extractors, and category classifiers.

import os
import sys
from langchain_core.tools import Tool
from langchain.agents import create_react_agent
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.types import Command
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate



# Go up one level to reach src/
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", "tools"))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

print(f"BASE_DIR: {BASE_DIR}")


# 🔁 Import all tools from registry
from tools.enhancer_tools_registry import (
    keyword_extractor_tool,
    extract_numeric_constraints_tool,
    filter_composer_tool
)

# Define tools for the enhancer agent
enhancer_tools = [
    keyword_extractor_tool,
    extract_numeric_constraints_tool,
    filter_composer_tool
]


# Define tool names for the agent
tool_names = [tool.name for tool in enhancer_tools]

# Define the tool descriptions
tool_descriptions = [tool.description for tool in enhancer_tools]

# Build readable tool help text for the prompt
tool_help_text = "\n".join(
    [f"{i+1}. {tool.name} - {tool.description}" for i, tool in enumerate(enhancer_tools)]
)


# Define system prompt used during agent creation

enhancer_agent_prompt_template = PromptTemplate.from_template(
    """You are a Query Enhancer Agent.

Your task is to:
1. Understand the user's query and its intent.
2. Clarify any ambiguities or vague instructions.
3. Add contextual details such as location, industry, amount, category, or other metadata if implied.
4. Output an improved version of the query, optimized for semantic and structured search.

Rules:
- Do not ask follow-up questions.
- Infer likely meanings when user input is ambiguous.
- Your output should be a standalone, precise query in natural language.
- Do not include explanations or formatting like “Here’s the improved query.”

Example:
1. Fuzzy Funding Reference → Structured Numeric + Stage Filter
User Query: "Startups with big funding"
Enhanced Prompt: "Indian startups that raised over $10 million in Series A or later funding rounds."
Focus: Adds monetary threshold and filters funding stage.

2. Location Mention → Structured Geographic Filter
User Query: "Top SaaS companies in Bangalore"
Enhanced Prompt: "Top-performing SaaS startups headquartered in Bengaluru with recent traction or funding activity."
Focus: Resolves city + domain, adds optional temporal or traction filter.

🔹 3. Team Size Mention → Employee Count Filter
User Query: "Companies with large teams working in AI"
Enhanced Prompt: "AI-focused startups in India with over 200 employees and active hiring status."
Focus: Converts “large team” into a numeric filter + hiring signal.

🔹 4. Recent Growth Mention → Year + Funding Filter
User Query: "Fast-growing e-commerce startups"
Enhanced Prompt: "E-commerce startups founded after 2018 with rapid growth and multiple funding rounds."
Focus: Infers growth using founding year and number of rounds.

🔹 5. Tech Stack Mention → Tool + Sector Filters
User Query: "Fintech companies using AWS and React"
Enhanced Prompt: "Fintech startups with a tech stack including AWS and React, offering scalable B2C solutions."
Focus: Extracts cloud/backend/frontend tech + sector.



You have access to the following tools:
{tools}

Format:
Question: the input query
Thought: think step-by-step about what to extract
Action: the tool to use, from [{tool_names}]
Action Input: JSON string or plain text input to the tool
Observation: result returned by the tool
... (repeat Thought/Action/Observation as needed)
Thought: I have gathered all necessary structured data.
Final Answer: a dictionary of all extracted metadata and filters

Constraints:
- NEVER ask the user again
- ONLY use tools
- NEVER hallucinate missing data

Begin!

Question: {input}
{agent_scratchpad}"""
)

# Format the prompt with tool descriptions and names
formatted_prompt = enhancer_agent_prompt_template.partial(
    tools=tool_help_text,
    tool_names=", ".join(tool.name for tool in enhancer_tools)
)

# 🔧 Define the React-style agent
llm = ChatOpenAI(model="gpt-4o")  # Or use your preferred model


# Create the agent
enhancer_agent = create_react_agent(
    llm=llm,
    tools=enhancer_tools,
    prompt=formatted_prompt,
    )


BASE_DIR: /home/prashant-agrawal/Netflix_Project/src/tools


In [21]:
def enhancer_node(state: dict) -> Command[Literal["supervisor"]]:
    """
    Enhancer node for refining and structuring vague user queries.

    Returns:
        Command: Enhanced message sent back to the supervisor with reasoning.
    """

    # 🧾 Instructional system message — passed as part of prompt to guide behavior
    system_prompt = (
        "You are an advanced query enhancer. Your task is to:\n"
        "- Clarify and refine vague or ambiguous user inputs.\n"
        "- Extract metadata using tools (industry, funding stage, filters).\n"
        "- NEVER generate final answers, only enhance the query.\n"
        "- Return your reasoning and structured results back to the supervisor."
    )

    # 🧠 Compose messages for the agent
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]

    # 🚀 Invoke the agent (you can use `invoke()` or stream if needed)
    enhanced_response = enhancer_agent.invoke(messages)

    print("🧩 Enhancer Node executed. Routing back to: supervisor")

    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=enhanced_response.content,
                    name="enhancer"
                )
            ]
        },
        goto="supervisor"
    )



In [22]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(agent=enhancer_agent, tools=enhancer_tools, verbose=True, handle_parsing_errors=True)

query = """

List D2C or SaaS companies in Delhi or Hyderabad that raised over ₹200 crore, are currently hiring for engineers and PMs, valued above $500 million, 

offer mobile apps or APIs, and are backed by Sequoia or Accel."""

result = executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...
Thought: I need to extract keywords related to company details such as location, tech stack, funding, valuation, hiring status, product offering, and investor support from the query. Additionally, I need to extract numeric constraints related to funding, valuation, and possibly hiring details.

Action: keyword_extractor
Action Input: "List D2C or SaaS companies in Delhi or Hyderabad that raised over ₹200 crore, are currently hiring for engineers and PMs, valued above $500 million, offer mobile apps or APIs, and are backed by Sequoia or Accel."
{'headquarters_city': ['delhi', 'hyderabad'], 'industry_sector': ['saas', 'd2c'], 'total_funding_raised_inr': ['raised', 'million', 'crore', '₹', 'cr', 'over ₹'], 'hiring_status': ['hiring', 'currently hiring'], 'popular_roles_open': ['engineers'], 'primary_products__services': ['mobile app'], 'investors': ['accel', 'sequoia']}Action: Numeric Constraint Extraction
Action Input: "List D2C or SaaS companies i